In [1]:
from geopy import distance

import pandas as pd

# Import Datasets

In [2]:
df_cities = pd.read_csv ('data/cities.csv')
df_cities.head(5)

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,admin,5213000,3934421
1,Montréal,45.500000,-73.583333,Canada,CA,Québec,NaN,3678000,2356556
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,NaN,2313328,603502
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,primary,1145000,812129
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,NaN,1110000,915322


In [3]:
df_weather_stations = pd.read_csv('data/climate.csv')
df_weather_stations.head(5)

,lng,lat,STATION_NAME,CLIMATE_IDENTIFIER,ID,LOCAL_DATE,PROVINCE_CODE,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,...,SPEED_MAX_GUST,SPEED_MAX_GUST_FLAG,COOLING_DEGREE_DAYS,COOLING_DEGREE_DAYS_FLAG,HEATING_DEGREE_DAYS,HEATING_DEGREE_DAYS_FLAG,MIN_REL_HUMIDITY,MIN_REL_HUMIDITY_FLAG,MAX_REL_HUMIDITY,MAX_REL_HUMIDITY_FLAG
0,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.1,2020-01-01 00:00:00,AB,2020,1,1,...,NaN,NaN,0.0,NaN,27.2,NaN,86.0,NaN,97.0,NaN
1,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.2,2020-01-02 00:00:00,AB,2020,1,2,...,53.0,NaN,0.0,NaN,25.9,NaN,47.0,NaN,99.0,NaN
2,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.3,2020-01-03 00:00:00,AB,2020,1,3,...,37.0,NaN,0.0,NaN,30.0,NaN,77.0,NaN,93.0,NaN
3,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.4,2020-01-04 00:00:00,AB,2020,1,4,...,65.0,NaN,0.0,NaN,23.1,NaN,64.0,NaN,92.0,NaN
4,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.5,2020-01-05 00:00:00,AB,2020,1,5,...,NaN,NaN,0.0,NaN,28.6,NaN,57.0,NaN,98.0,NaN


# Exploratory Data Analysis

### Missing Values

From page 23 in https://climate.weather.gc.ca/doc/Technical_Documentation.pdf, I learned that "M" in temperature flag refers to "Missing".

In [4]:
print("The number of rows in the raw weather stations dataframe is {}".format(len(df_weather_stations)))
print("The number of rows in the raw weather stations dataframe where mean temperature is not none is {}".format(len(df_weather_stations[df_weather_stations['MEAN_TEMPERATURE'].notnull()])))
print("The number of rows in the raw weather stations dataframe where mean temperature is missing is {}".format(len(df_weather_stations[df_weather_stations['MEAN_TEMPERATURE_FLAG'] == "M"])))
print("The number of rows in the raw weather stations dataframe where min temperature is not none is {}".format(len(df_weather_stations[df_weather_stations['MIN_TEMPERATURE'].notnull()])))
print("The number of rows in the raw weather stations dataframe where min temperature is missing is {}".format(len(df_weather_stations[df_weather_stations['MIN_TEMPERATURE_FLAG'] == "M"])))
print("The number of rows in the raw weather stations dataframe where max temperature is not none is {}".format(len(df_weather_stations[df_weather_stations['MAX_TEMPERATURE'].notnull()])))
print("The number of rows in the raw weather stations dataframe where max temperature is missing is {}".format(len(df_weather_stations[df_weather_stations['MAX_TEMPERATURE_FLAG'] == "M"])))

The number of rows in the raw weather stations dataframe is 41513
The number of rows in the raw weather stations dataframe where mean temperature is not none is 39456
The number of rows in the raw weather stations dataframe where mean temperature is missing is 1643
The number of rows in the raw weather stations dataframe where min temperature is not none is 39573
The number of rows in the raw weather stations dataframe where min temperature is missing is 1526
The number of rows in the raw weather stations dataframe where max temperature is not none is 39538
The number of rows in the raw weather stations dataframe where max temperature is missing is 1561


We will only keep data points that provide mean, min and max temperature readings for simplicity purpose.

In [5]:
df_weather_stations = df_weather_stations[df_weather_stations['MEAN_TEMPERATURE'].notnull() & (df_weather_stations['MEAN_TEMPERATURE_FLAG'] != "M") & df_weather_stations['MAX_TEMPERATURE'].notnull() & (df_weather_stations['MAX_TEMPERATURE_FLAG'] != "M") & df_weather_stations['MIN_TEMPERATURE'].notnull() & (df_weather_stations['MIN_TEMPERATURE_FLAG'] != "M")]

We will only keep information that is relevant to this specific tasks.

In [6]:
df_cities = df_cities[["city", "lat", "lng", "admin"]]
df_weather_stations = df_weather_stations[["lng", "lat", "LOCAL_YEAR", "LOCAL_MONTH", "LOCAL_DAY", "MEAN_TEMPERATURE", "MIN_TEMPERATURE", "MAX_TEMPERATURE"]]

In [7]:
df_weather_stations.describe()

,lng,lat,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,MEAN_TEMPERATURE,MIN_TEMPERATURE,MAX_TEMPERATURE
count,39456.000000,39456.000000,39456.0,39456.000000,39456.000000,39456.000000,39456.000000,39456.000000
mean,-99.089468,52.455350,2020.0,1.327023,13.400953,-11.061200,-15.370225,-6.741266
std,23.057372,6.721156,0.0,0.469132,8.614395,11.507346,12.233960,11.262908
min,-140.868889,41.948614,2020.0,1.000000,1.000000,-50.000000,-51.800000,-48.100000
25%,-117.806111,48.667222,2020.0,1.000000,6.000000,-19.200000,-24.900000,-13.900000
50%,-105.250000,50.465000,2020.0,1.000000,12.000000,-8.700000,-13.600000,-3.700000
75%,-78.835556,54.295920,2020.0,2.000000,20.000000,-2.300000,-5.700000,1.500000
max,-52.783339,76.425833,2020.0,2.000000,31.000000,11.500000,9.800000,16.500000


# Combine Two Datasets

In [8]:
print("The number of data points that the latitude and longitude in both datasets are overalpped is {}".format(len(pd.merge(df_cities, df_weather_stations,  how='inner', on=['lng','lat']))))

The number of data points that the latitude and longitude in both datasets overalpped is 0


Becaue no weather stations' locations are perfectly matched with the locations from the cities dataset. We have to calculate the distance between each weather station and each city first.

In [19]:
df_distinct_weather_stations_locations = df_weather_stations.groupby(['lng', 'lat']).size().reset_index(name='Freq').sort_values(["lng", "lat"]).drop('Freq', axis=1, inplace=False)
df_distinct_weather_stations_locations.head(3)

,lng,lat
0,-140.868889,62.410278
1,-140.847500,68.800556
2,-140.150000,69.164444


In [20]:
df_distinct_cities_locations = df_cities.groupby(['lng', 'lat']).size().reset_index(name='Freq').sort_values(["lng", "lat"]).drop('Freq', axis=1, inplace=False)
df_distinct_cities_locations.head(3)

,lng,lat
0,-139.45,64.066667
1,-139.00,61.350000
2,-135.05,60.716667


Next, we want to find the closest city for each weather station.

In [26]:
df_cartesian_locations = df_distinct_cities_locations.assign(key=1).merge(df_distinct_weather_stations_locations.assign(key=1), on="key").drop("key", axis=1)
df_cartesian_locations.head(3)

,lng_x,lat_x,lng_y,lat_y
0,-139.45,64.066667,-140.868889,62.410278
1,-139.45,64.066667,-140.847500,68.800556
2,-139.45,64.066667,-140.150000,69.164444


In [27]:
df_cartesian_locations.columns = ['lng_city', 'lat_city', 'lng_station', 'lat_station']
df_cartesian_locations.head(3)

,lng_city,lat_city,lng_station,lat_station
0,-139.45,64.066667,-140.868889,62.410278
1,-139.45,64.066667,-140.847500,68.800556
2,-139.45,64.066667,-140.150000,69.164444


In [29]:
def calculate_distance(lat_x, lng_x, lat_y, lng_y):
    coords_1 = (lat_x, lng_x) # lat, lng
    coords_2 = (lat_y, lng_y)
    return distance.geodesic(coords_1, coords_2).km

df_cartesian_locations['distance'] = df_cartesian_locations.apply(lambda x: calculate_distance(x.lat_city, x.lng_city, x.lat_station, x.lng_station), axis=1)

In [30]:
df_cartesian_locations.head(3)

,lng_city,lat_city,lng_station,lat_station,distance
0,-139.45,64.066667,-140.868889,62.410278,197.911383
1,-139.45,64.066667,-140.847500,68.800556,531.528585
2,-139.45,64.066667,-140.150000,69.164444,569.318843


We want to find the nearest city to each weather station.

In [34]:
df_distance = df_cartesian_locations.sort_values("distance").groupby(['lng_station','lat_station'], as_index=False).first()
df_distance.head(3)

,lng_station,lat_station,lng_city,lat_city,distance
0,-140.868889,62.410278,-139.000000,61.350000,153.698638
1,-140.847500,68.800556,-134.866667,67.433333,291.689404
2,-140.150000,69.164444,-133.700000,68.350000,276.117613


In [35]:
df_distance['distance'].describe()

count    898.000000
mean      58.425812
std       65.929424
min        0.000021
25%        7.880352
50%       40.722375
75%       83.086362
max      616.310421
Name: distance, dtype: float64

From https://www.researchgate.net/post/Data-and-weather-station-distance, I learned that I can apply a distance threshold ~15KM to filter out rows that contain cities and weather stations that are fat away from each other.

In [59]:
df_close_locations = df_distance[df_distance['distance'] <= 15]
df_close_locations.head(3)

,lng_station,lat_station,lng_city,lat_city,distance
7,-139.040000,61.370556,-139.00,61.350000,3.134549
19,-135.097778,60.733056,-135.05,60.716667,3.183310
20,-135.067222,60.709444,-135.05,60.716667,1.237624


In [60]:
df_stations_cities = pd.merge(df_close_locations, df_cities,  how='left', left_on = ['lng_city','lat_city'], right_on = ['lng','lat']).drop(['lng','lat'], axis=1, inplace=False)
df_stations_cities.head(3)

,lng_station,lat_station,lng_city,lat_city,distance,city,admin
0,-139.040000,61.370556,-139.00,61.350000,3.134549,Burwash Landing,Yukon
1,-135.097778,60.733056,-135.05,60.716667,3.183310,Whitehorse,Yukon
2,-135.067222,60.709444,-135.05,60.716667,1.237624,Whitehorse,Yukon


In [61]:
df_stations_cities = pd.merge(df_stations_cities, df_weather_stations,  how='right', left_on = ['lng_station','lat_station'], right_on = ['lng','lat']).drop(['lng','lat', 'lng_station', 'lat_station', 'lng_city', 'lat_city', 'distance'], axis=1, inplace=False)
df_stations_cities.head(3)

,city,admin,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,MEAN_TEMPERATURE,MIN_TEMPERATURE,MAX_TEMPERATURE
0,NaN,NaN,2020,1,1,-9.2,-16.4,-2.1
1,NaN,NaN,2020,1,2,-7.9,-14.7,-1.1
2,NaN,NaN,2020,1,3,-12.0,-17.8,-6.2


We will only keep records that are close to cities.

In [62]:
df_stations_cities = df_stations_cities.dropna(how='any')
df_stations_cities.head(3)

,city,admin,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,MEAN_TEMPERATURE,MIN_TEMPERATURE,MAX_TEMPERATURE
227,Camrose,Alberta,2020,1,1,-5.6,-11.9,0.7
228,Camrose,Alberta,2020,1,2,-6.3,-12.0,-0.6
229,Camrose,Alberta,2020,1,3,-7.1,-10.3,-3.8


In [77]:
def get_temperature(year=2020, month=1, day=1):
    df = df_stations_cities[(df_stations_cities['LOCAL_YEAR'] == year) & (df_stations_cities['LOCAL_MONTH'] == month) & (df_stations_cities['LOCAL_DAY'] == day)]
    
    df_mean = df.groupby(['city', 'admin'], as_index=False).agg({'MEAN_TEMPERATURE':'mean', 'MIN_TEMPERATURE':'mean','MAX_TEMPERATURE':'mean'}) \
       .rename(columns={'city':'City', 'admin':'Province', 'MEAN_TEMPERATURE':'MEAN_TEMPERATURE_MEAN', 'MIN_TEMPERATURE':'MIN_TEMPERATURE_MEAN','MAX_TEMPERATURE':'MAX_TEMPERATURE_MEAN'})
    
    df_median = df.groupby(['city', 'admin'], as_index=False).agg({'MEAN_TEMPERATURE':'median', 'MIN_TEMPERATURE':'median','MAX_TEMPERATURE':'median'}) \
       .rename(columns={'city':'City', 'admin':'Province', 'MEAN_TEMPERATURE':'MEAN_TEMPERATURE_MEDIAN', 'MIN_TEMPERATURE':'MIN_TEMPERATURE_MEDIAN','MAX_TEMPERATURE':'MAX_TEMPERATURE_MEDIAN'})
    return df_mean, df_median

In [78]:
df_mean, df_median = get_temperature(year=2020, month=1, day=1)

In [79]:
df_mean.head(3)

,City,Province,MEAN_TEMPERATURE_MEAN,MIN_TEMPERATURE_MEAN,MAX_TEMPERATURE_MEAN
0,Abbotsford,British Columbia,8.8,6.0,11.5
1,Amherst,Nova Scotia,1.8,-0.9,4.6
2,Antigonish,Nova Scotia,1.0,-1.2,3.2


In [81]:
df_median.head(3)

,City,Province,MEAN_TEMPERATURE_MEDIAN,MIN_TEMPERATURE_MEDIAN,MAX_TEMPERATURE_MEDIAN
0,Abbotsford,British Columbia,8.8,6.0,11.5
1,Amherst,Nova Scotia,1.8,-0.9,4.6
2,Antigonish,Nova Scotia,1.0,-1.2,3.2


# Conclusion

Trade-offs and Assumptions:

I assumed that I only use the data points have all three temperature readings. An advantage is that we don't need to deal with missing values in later computation. The disadvantage is that we have less data to work with.

Another assumption that I made is that the record from weather stations is only effective within 15KM. This drastically reduces the number of data points. But this give us more accurate results in the later stage.

Scalability:
    
When I merge the cities dataset and weather stations dataset, I did a cartesian product before I calcualte distance. This will be a huge bottleneck when there are a lot of cities and a lot of weather stations. Assume there are a lot more weather stations in the dataset, a better approach is to compute distance between each weather station and all cities.